In [10]:
from IPython.display import display, HTML, clear_output, Image
import time
import ipywidgets as widgets
import random 
import pandas as pd
from jupyter_ui_poll import ui_events
import requests
from bs4 import BeautifulSoup
import json
import ANSQuestion_generator as ag




# obtain consent from participants 
data_consent_info = """DATA CONSENT INFORMATION:

Please read:

We wish to record your response data
to an anonymised public data repository. 
Your data will be used for educational teaching purposes
practising data analysis and visualisation.

Please type   yes   in the box below if you consent to the upload."""

print(data_consent_info)
result = input("> ") 

if result == "yes": 
    print("Thanks for your participation.")
    print("Please contact a.fedorec@ucl.ac.uk")
    print("If you have any questions or concerns")
    print("regarding the stored results.")
    
else: 
    # end code execution by raising an exception
    raise(Exception("User did not consent to continue test."))

# test introduction
html_1 = HTML("<span style='color:red; font_size: 20px';>Welcome to the ANS test!</span>")
display(html_1)

# collect some basic personal information
id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise
     then your unique identifier would be CBTC
"""

print(id_instructions)
user_id = input("> ")

print("User entered id:", user_id)

participant_id = user_id
participant_gender = input('What is your gender? Please input Female or Male')
participant_age = input('What is your age? Please input a number')

# data dictionary with photos and their correponding information, including dots number, correct answer, and ratio
output_dict = ag.ANSQuestion_bank(12)

# Initialise counters
correct_num = 0
incorrect_num = 0

# Event info dictionary to capture button click events
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

# data dictionary used to collect final results
data_dict = {
    'id':[],
    'gender': [],
    'age':[],
    'score': [],
    'filename': [],
    'num_L':[],
    'num_R':[],
    'ratio':[],
    'correct_answer':[],
    'correctness':[],
    'time_taken':[],
}

def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok


# Function to wait for a button click event or timeout
def wait_for_event(timeout=3, interval=0.1, max_rate=20, allow_interupt=True):
    start_wait = time.time()

    # Reset event info
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate * interval) + 1

    with ui_events() as ui_poll:
        while True:
            ui_poll(n_proc)

            # Check for timeout
            if (timeout != -1) and (time.time() > start_wait + timeout):
                break
        
            # Check if an event has occurred
            if allow_interupt and event_info['description'] != "":
                break
        
            time.sleep(interval)
    return

# Register a button click event
def register_event(btn):
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

# Display each trial and wait for response check
def trial(png,i,idx):
    # create correctness to check the correctness of each question and a correct_answer list for later use
    global correct_num, incorrect_num
    correctness = 0
    correct_answer = output_dict[i][0]

    clear_output(wait=True)

    # display a float which can show the progress of thw whole test
    display(widgets.FloatProgress(
        value=1+idx,
        min=0,
        max=64,
        description='progessing:',
        bar_style='info',
        orientation='horizontal'
    ))

    # display the test images 
    display(Image(png, width=300))
    time.sleep(0.75)

    clear_output(wait=True)

    # display the progress bar, images without dots, questions, and buttons to click on
    display(widgets.FloatProgress(
        value=1+idx,
        min=0,
        max=64,
        description='progessing:',
        bar_style='info',
        orientation='horizontal'
    ))
    display(Image('./Figures/fixation_cross.png', width=300))

    display(HTML("<span style='color:red; font-size: 30px;'>Which side has more dots? left or right?</span>"))
    
    btn_left = widgets.Button(description="left")
    btn_right = widgets.Button(description="right")
    btn_left.on_click(register_event)
    btn_right.on_click(register_event)
    display(widgets.HBox([btn_left, btn_right]))
    
    wait_for_event(timeout=3)

    # check the correctness of response by using correct_answer
    response = event_info['description']
    if response == correct_answer:
        correct_num += 1
        correctness += 1 
    else:
        incorrect_num += 1
        correctness = 0
    data_dict['correctness'].append(correctness)
    return

# run the whole test
idx = 0
for i in output_dict:
    start_time = time.time()
    clear_output
    png = i   
    trial(png,i,idx)
    clear_output()
    end_time = time.time()
    time.sleep(1.5)
    test_time = end_time - start_time
    # add results of each trial to final data dictionary
    global results_dict
    data_dict['filename'].append(png)
    data_dict['correct_answer'].append(output_dict[i][0])
    data_dict['num_L'].append(output_dict[i][1])
    data_dict['num_R'].append(output_dict[i][2])
    data_dict['ratio'].append(output_dict[i][3])
    data_dict['time_taken'].append(test_time)
    idx +=1

# collect both the personal information and final test score into data_dict
data_dict['id'].append(participant_id)
data_dict['gender'].append(participant_gender)
data_dict['age'].append(participant_age)
data_dict['score'].append(correct_num)

# transfer data into google form
form_url = 'https://docs.google.com/forms/d/e/1FAIpQLScXc70t6YTFENnG4Y0gYerZ1SbjovZ-e3xHSkx_tMdzMqoasQ/viewform?usp=sf_link'
send_to_google_form(data_dict, form_url)

# show the final results to participants
print(f'Your correct number is: {correct_num}.')


Your correct number is: 57.
